In [ ]:
import tifffile
import euler_gpu
import os, random
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import matplotlib.pyplot as plt
import numpy as np
import torch
import h5py
from tqdm import tqdm
device = torch.device("cuda:0")

### These should be the only things you need to change ###
tiff_folder = "/storage/fs/store1/brian/swiming_videos/Folder_20240930105926_good"

BATCH_SIZE = 128

# How many frames to average over when generating the transformation. All frames will be aligned.
num_frames = 128

# The two channels to align. Green should correspond to the one that is variable
red_channel = 0
green_channel = 1

# Since the green channel can be highly variant and have large maximum values, it can throw off the NCC calculation after normalization.
# I have found that caping the maximum value of the green channel to 250 works well, although you could also take a log transformation of that channel.
# It's likely that 250 might not be a good value for all datasets, so if there's an issue try changing this value.
cutoff = 250

## Align Entire Video

In [ ]:
tiff_files = sorted([f for f in os.listdir(tiff_folder) if os.path.splitext(f)[-1] == ".tif" and 
                     (f.find("RIG") < 0 and f.find("WARP") < 0 and f.find("CHAN") < 0)], key=lambda x: int(x.split('_')[-1].split('-')[0])) 
tiff_files = tiff_files[10:]
print(tiff_files)


['20240930_20000-21999.tif', '20240930_22000-23999.tif', '20240930_24000-25999.tif', '20240930_26000-27999.tif', '20240930_28000-29999.tif']


In [ ]:
ALIGN_XY_RANGE = np.linspace(-0.001, 0.001, 20, dtype=np.float32)
# ALIGN_TH_RANGE = np.concatenate((np.linspace(0, .1, 10, dtype=np.float32), np.linspace(359.9, 360, 10, dtype=np.float32)))
ALIGN_TH_RANGE = np.concatenate((np.linspace(0, .1, 10, dtype=np.float32), np.linspace(-0.1, 0, 10, dtype=np.float32)))

#### Generate an optimal transformation randomly sampling from frames

In [ ]:
x_tot = 0.0
y_tot = 0.0
th_tot = 0.0


ins = tifffile.imread(os.path.join(tiff_folder, tiff_files[0]))
frames = random.sample(range(ins.shape[0]), num_frames)
for frame in tqdm(frames):
    red_image = ins[frame, red_channel]
    green_image = ins[frame, green_channel]
    green_image_log = np.minimum(green_image, cutoff)

    memory_dict = euler_gpu.initialize(red_image, green_image_log, ALIGN_XY_RANGE, ALIGN_XY_RANGE, ALIGN_TH_RANGE, BATCH_SIZE, device)
    best_score, best_transformation = euler_gpu.grid_search(memory_dict)

    x_tot += best_transformation[0]
    y_tot += best_transformation[1]
    th_tot += best_transformation[2]

print(x_tot/num_frames)
print(y_tot/num_frames)
print(th_tot/num_frames)


100%|██████████| 20/20 [02:38<00:00,  7.94s/it]

tensor([0.0001], device='cuda:0')
tensor([-0.0006], device='cuda:0')
tensor([6.9813e-05], device='cuda:0')


In [9]:
for file in tqdm(tiff_files):
    ins = tifffile.imread(os.path.join(tiff_folder, file))
    memory_dict = euler_gpu.initialize(ins[0,0], ins[0,0], ALIGN_XY_RANGE, ALIGN_XY_RANGE, ALIGN_TH_RANGE, BATCH_SIZE, device)

    for frame in range(ins.shape[0]):
        in_img = ins[frame, green_channel]
        in_img = torch.Tensor(in_img[np.newaxis, np.newaxis, ...]).to(device=device)
        ins[frame, green_channel] = euler_gpu.transform_image(in_img, best_transformation[0], best_transformation[1], best_transformation[2], memory_dict).cpu().numpy()
    tifffile.imwrite(os.path.join(tiff_folder, "CHAN_" + file), ins, imagej=True)

  0%|          | 0/5 [00:00<?, ?it/s]/home/brian/.conda/envs/PB/lib/python3.11/site-packages/tifffile/tifffile.py:3784: UserWarning: <tifffile.TiffWriter 'CHAN_20240930_20000-21999.tif'> truncating ImageJ file
  warnings.warn(
 20%|██        | 1/5 [02:01<08:04, 121.16s/it]/home/brian/.conda/envs/PB/lib/python3.11/site-packages/tifffile/tifffile.py:3784: UserWarning: <tifffile.TiffWriter 'CHAN_20240930_22000-23999.tif'> truncating ImageJ file
  warnings.warn(
 40%|████      | 2/5 [05:49<09:12, 184.25s/it]/home/brian/.conda/envs/PB/lib/python3.11/site-packages/tifffile/tifffile.py:3784: UserWarning: <tifffile.TiffWriter 'CHAN_20240930_24000-25999.tif'> truncating ImageJ file
  warnings.warn(
 60%|██████    | 3/5 [10:02<07:11, 215.65s/it]/home/brian/.conda/envs/PB/lib/python3.11/site-packages/tifffile/tifffile.py:3784: UserWarning: <tifffile.TiffWriter 'CHAN_20240930_26000-27999.tif'> truncating ImageJ file
  warnings.warn(
 80%|████████  | 4/5 [13:11<03:25, 205.19s/it]/home/brian/.conda/